In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
import seaborn as sns
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
import time
import datetime
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import numpy as np, pandas as pd, os, gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [ ]:
def detect_num_cols_to_shrink(list_of_num_cols, dataframe):
 
    convert_to_int8 = []
    convert_to_int16 = []
    convert_to_int32 = []
    
    #  sadly the datatype float8 does not exist
    convert_to_float16 = []
    convert_to_float32 = []
    
    for col in list_of_num_cols:
        
        if dataframe[col].dtype in ['int', 'int8', 'int32', 'int64']:
            
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 255:
                convert_to_int8.append(col)
                
            elif diff < 65535:
                convert_to_int16.append(col)
                
            elif diff < 4294967295:
                convert_to_int32.append(col)   
                
        elif dataframe[col].dtype in ['float', 'float16', 'float32', 'float64']:
            
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 65535:
                convert_to_float16.append(col)
                
            elif diff < 4294967295:
                convert_to_float32.append(col) 
        
    list_of_lists = []
    list_of_lists.append(convert_to_int8)
    list_of_lists.append(convert_to_int16)
    list_of_lists.append(convert_to_int32)
    list_of_lists.append(convert_to_float16)
    list_of_lists.append(convert_to_float32)
    
    return list_of_lists

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train_transaction = reduce_mem_usage(train_transaction, verbose = True)
train_identity = reduce_mem_usage(train_identity,verbose = True)
test_transaction = reduce_mem_usage(test_transaction, verbose=True)
test_identity = reduce_mem_usage(test_identity,verbose = True)

In [ ]:
pd.set_option('display.max_columns', None)

Tổng quát:
- **1. GIỚI THIỆU DATA:**
    - 1.1 Thông tin tổng quát dữ liệu
    - 1.2 Thông tin tổng quát về Target
    - 1.3 Metric đánh giá
- **2. KHAI PHÁ DỮ LIỆU**
    - 2.1 Dữ liệu Numerical
    - 2.2 Dữ liệu Categorical
    - 2.3 Tổng hợp các dữ liệu lấy làm Feature
- **3. FEATURE ENGINEERING**
    - 3.1 Xử lý dữ liệu numerical
    - 3.2 Xử lý dữ liệu Categorical
- **4. XÂY DỰNG MÔ HÌNH**

# **1. GIỚI THIỆU DATA**

- **1.1 Tổng quát dữ liệu vô**

- train_identity:
 - 23 loại Numerical
 - 14 categorical
 - 4 Boolean
 - Tổng quan: Nhiều biến thiếu dữ liệu

- train_transaction:
 - 356 loại Numerical @@ (30/6/2021 chưa sửa)
 - 30 categorical
 - 8 loại boolean
 - Tổng quan: File hết 1.7gb, quá nhiều biến, cần giảm chiều dữ liệu
 - Giảm chiều ntn chưa biết

In [ ]:
train_transaction.head()

In [ ]:
train_identity.head()

In [ ]:
test_transaction.head()

In [ ]:
test_identity.head()

In [ ]:
#gom 2 dataset lại thành 1 theo transactionID
final_train = pd.merge(train_transaction, train_identity, how = 'left', on = 'TransactionID')

final_test = pd.merge(test_transaction, test_identity, how = 'left', on = 'TransactionID')
#final_test = test_transaction.merge(test_identity, on='TransactionID', how = 'inner')


In [ ]:
del train_transaction
del train_identity
del test_transaction
del test_identity

In [ ]:
print(final_train.shape)
print(final_test.shape)

 **Kiểm tra giữa 2 file có các cột nào khác nhau**

We have observed there exist difference style in column name. Therefore, need to solve this issue.

In [ ]:
def different_columns(traincols, testcols):
    
    for i in traincols:
        
        if i not in testcols:
            
            print(i)
            
different_columns(final_train.columns, final_test.columns)

In [ ]:
final_test = final_test.rename(columns = {"id-01": "id_01", "id-02": "id_02", "id-03": "id_03", 
                            "id-06": "id_06", "id-05": "id_05", "id-04": "id_04", 
                            "id-07": "id_07", "id-08": "id_08", "id-09": "id_09", 
                            "id-10": "id_10", "id-11": "id_11", "id-12": "id_12", 
                            "id-15": "id_15", "id-14": "id_14", "id-13": "id_13", 
                            "id-16": "id_16", "id-17": "id_17", "id-18": "id_18", 
                            "id-21": "id_21", "id-20": "id_20", "id-19": "id_19", 
                            "id-22": "id_22", "id-23": "id_23", "id-24": "id_24", 
                            "id-27": "id_27", "id-26": "id_26", "id-25": "id_25", 
                            "id-28": "id_28", "id-29": "id_29", "id-30": "id_30", 
                            "id-31": "id_31", "id-32": "id_32", "id-33": "id_33", 
                            "id-34": "id_34", "id-35": "id_35", "id-36": "id_36", 
                            "id-37": "id_37", "id-38": "id_38"})
final_test.head()

**Kiểm tra lại train và test có còn cột khác không**

In [ ]:
different_columns(final_train.columns, final_test.columns)

### **1.2 Thông tin tổng quát Target**

In [ ]:
y = final_train.isFraud
y.describe
#1 là fraud, còn 0 là k phải fraud
sns.displot(
    data=y
)
#dữ liệu khá phân bố sai, xử lý ntn

In [ ]:
y.dtypes

 **1.2 Thông tin tổng quát Metric**

 - Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.
 - Nên sử dụng F1 score, tìm Precision, Recall, Confusion Matrix
 - Nên dùng model nào để dễ tối ưu AUC???




# **2. KHAI PHÁ DỮ LIỆU**

### **Check Nan trong Data**

In [ ]:
def getNulls(data):
    
    total = data.isnull().sum()
    percent = data.isnull().sum() / data.isnull().count()
    missing_data = pd.concat([total, percent], axis = 1, keys = ['total', 'precent'])
    
    return missing_data

In [ ]:
missing_data_train = getNulls(final_train)
missing_data_train.head(434).T

### **Tổng kết về Nan:** ###
- Nhiều cột có Nan hơn 25% **=>** Nếu bỏ thì sẽ mất khá nhiều data
- Ít dữ liệu nên **fill Nan bằng số ảo** (-999 hoặc -1)
- Có nhiều cột **mang thông tin đặc trưng** (card1-6 là số thẻ, addr1-2 là số nhà). **Fill bằng mean sẽ không có ý nghĩa**

In [ ]:
del missing_data_train

## **2.1 Dữ liệu Numerical**:

In [ ]:
numerical = final_train.select_dtypes(include='number')
numerical.head()

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc. 
- addr: address
- dist: distance
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked. 
- D1-D15: timedelta, such as days between previous transaction, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

### **Vesta features**: ###
The V columns share a “lot” of correlation and a large number of Nan’s. The goal of this step is to find “similar” columns based on number of “NaN’s”, and Correlation>0.75. This process is automated for all the different NaN groups, using a script. The method is explained as below:

In [ ]:
def make_corr(Vs,Vtitle=''):
    cols = ['isFraud'] + Vs
    plt.figure(figsize=(15,15))
    sns.heatmap(numerical[cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
    if Vtitle!='': plt.title(Vtitle,fontsize=14)
    else: plt.title(Vs[0]+' - '+Vs[-1],fontsize=14)
    plt.show()
Vs = ['V'+str(i) for i in range(5,16)]
Vtitle = 'V5 - V16'
make_corr(Vs,Vtitle)

### **Card1, 2, 3, 5:** ###

In [ ]:
sns.distplot(a = numerical['card1'])

### **TransactionAmt**: ###

In [ ]:
numerical['TransactionAmt'].describe()

In [ ]:
sns.distplot(a = numerical['TransactionAmt'])

Xử dụng Log1P để đưa về normal distribution

In [ ]:
sns.distplot(a= np.log1p(numerical['TransactionAmt']))

### **Distance**: ###
distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.

## **2.2 Dữ liệu Categorical**

In [ ]:
#coi thử categorical
categorical = final_train.select_dtypes(exclude='number')
categorical

- ProductCD: product code, the product for each transaction 
- card4, card6: payment card information, such as card type, card category, issue bank, country, etc. 
- M1-M9: match, such as names on card and address, etc.
- idxx, DeviceType, DeviceInfo: Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions. 


## **2.3 Dữ liệu Boolean**

# **3. Feature Engineering**

In [ ]:
ntrain = final_train.shape[0]
ntest = final_test.shape[0]
all_data = pd.concat([final_train, final_test], axis = 0, sort = False)

In [ ]:
all_data = all_data.drop(columns=['isFraud'])

In [ ]:
del numerical
del categorical
del final_train
del final_test

In [ ]:
all_data.shape

In [ ]:
all_data_cols = all_data.columns

In [ ]:
n = (all_data.dtypes != 'object')

num_all_cols = list(n[n].index) 

print(num_all_cols)

### **Giảm kích thước dữ liệu**

In [ ]:
num_cols_to_shrink_all = detect_num_cols_to_shrink(num_all_cols, all_data)

convert_to_int8 = num_cols_to_shrink_all[0]
convert_to_int16 = num_cols_to_shrink_all[1]
convert_to_int32 = num_cols_to_shrink_all[2]

convert_to_float16 = num_cols_to_shrink_all[3]
convert_to_float32 = num_cols_to_shrink_all[4]

In [ ]:
print("starting with converting process....")

for col in convert_to_int16:
    
    all_data[col] = all_data[col].astype('int16') 
    
for col in convert_to_int32:
    all_data[col] = all_data[col].astype('int32') 

for col in convert_to_float16:
    all_data[col] = all_data[col].astype('float16')
    
for col in convert_to_float32:
    all_data[col] = all_data[col].astype('float32')
    
print("successfully converted!")

### **Chọn Features**

In [ ]:
all_data

In [ ]:
v =  [1, 3, 4, 6, 8, 11]
v += [13, 14, 17, 20, 23, 26, 27, 30]
v += [36, 37, 40, 41, 44, 47, 48]
v += [54, 56, 59, 62, 65, 67, 68, 70]
v += [76, 78, 80, 82, 86, 88, 89, 91]

v += [96, 98, 99, 104] #relates to groups, no NAN 
v += [107, 108, 111, 115, 117, 120, 121, 123] # maybe group, no NAN
v += [124, 127, 129, 130, 136] # relates to groups, no NAN

# LOTS OF NAN BELOW
v += [138, 139, 142, 147, 156, 162] #b1
v += [165, 160, 166] #b1
v += [178, 176, 173, 182] #b2
v += [187, 203, 205, 207, 215] #b2
v += [169, 171, 175, 180, 185, 188, 198, 210, 209] #b2
v += [218, 223, 224, 226, 228, 229, 235] #b3
v += [240, 258, 257, 253, 252, 260, 261] #b3
v += [264, 266, 267, 274, 277] #b3
v += [220, 221, 234, 238, 250, 271] #b3

v += [294, 284, 285, 286, 291, 297] # relates to grous, no NAN
v += [303, 305, 307, 309, 310, 320] # relates to groups, no NAN
v += [281, 283, 289, 296, 301, 314] # relates to groups, no NAN
v += [332, 325, 335, 338] # b4 lots NAN

In [ ]:
cols = ['V'+str(x) for x in v]
for i in all_data_cols:
    if (i.startswith("V")) and i not in cols:
        all_data = all_data.drop(columns=[i])

all_data     

In [ ]:
# FAILED TIME CONSISTENCY TEST
for c in ['C3','M5','id_08','id_33']:
    cols.remove(c)
    all_data = all_data.drop(columns=[i])
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    cols.remove(c)
    all_data = all_data.drop(columns=[i])
for c in ['id_'+str(x) for x in range(22,28)]:
    cols.remove(c)
    all_data = all_data.drop(columns=[i])
for c in ['D6','D7','D8','D9','D12','D13','D14']:
    cols.remove(c)
    all_data = all_data.drop(columns=[i])

### **Pipeline Preprocessing**

In [ ]:
numerical = all_data.select_dtypes(include='number')
categorical = all_data.select_dtypes(exclude = 'number')

numeric_transformer = Pipeline(steps=[('mean',SimpleImputer(strategy='constant',fill_value=-1))])
categorical_transformer = Pipeline(steps=[('constant', SimpleImputer(strategy='constant',fill_value=-1))])

In [ ]:
#final_all_data[numerical.columns]= numeric_transformer.fit_transform(final_all_data[numerical.columns])
#final_all_data[categorical.columns] = categorical_transformer.fit_transform(final_all_data[categorical.columns])

In [ ]:
final_train = all_data[ : ntrain]
final_test = all_data[ntrain : ]

In [ ]:
del all_data

### **FACTORIZE CATEGORICAL VARIABLES and SHIFT ALL NUMERICS POSITIVE SET NAN to -1**

In [ ]:
for i,f in enumerate(final_train.columns):
    # FACTORIZE CATEGORICAL VARIABLES
    if (np.str(final_train[f].dtype)=='category')|(final_train[f].dtype=='object'): 
        df_comb = pd.concat([final_train[f],final_test[f]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        if df_comb.max()>32000: print(f,'needs int32')
        final_train[f] = df_comb[:len(final_train)].astype('int16')
        final_test[f] = df_comb[len(final_train):].astype('int16')
    # SHIFT ALL NUMERICS POSITIVE. SET NAN to -1
    elif f not in ['TransactionAmt','TransactionDT']:
        mn = np.min((final_train[f].min(),final_test[f].min()))
        final_train[f] -= np.float32(mn)
        final_test[f] -= np.float32(mn)
        final_train[f].fillna(-999,inplace=True)
        final_test[f].fillna(-999,inplace=True)

In [ ]:
y.value_counts()

In [ ]:
final_train.head()

In [ ]:
final_test.head()

# **4. Xây dựng Model**

In [ ]:
#sử dụng CatBoostClassifier regressor
#model = LogisticRegression(solver='lbfgs',max_iter=10000)

#model = CatBoostClassifier()
model = xgb.XGBClassifier(n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        #nthread=4,
        tree_method='hist' 
        #tree_method='gpu_hist' 
                           )

In [ ]:
X_train = final_train
y_train = y
X_test = final_test

In [ ]:
del final_train
del y
del final_test

In [ ]:
final_col = X_train['TransactionID']
X_train = X_train.drop(columns=['TransactionID'])

In [ ]:
final_col

## **No UID**

In [ ]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
X_train['DT_M'] = X_train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_train['DT_M'] = (X_train['DT_M'].dt.year-2017)*12 + X_train['DT_M'].dt.month 
X_test['DT_M'] = X_test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_test['DT_M'] = (X_test['DT_M'].dt.year-2017)*12 + X_test['DT_M'].dt.month 

In [ ]:
cols = list(X_train.columns )
cols.remove('TransactionDT')

In [ ]:
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

In [ ]:
X_train

In [ ]:
#h = model.fit(X_train.loc[idxT,cols], y_train[idxT], 
        #eval_set=[(X_train.loc[idxV,cols],y_train[idxV])],
       # verbose=50, early_stopping_rounds=100)

In [ ]:
cols

In [ ]:
oof = np.zeros(len(X_train))
preds = np.zeros(len(X_test))
skf = GroupKFold(n_splits=6)
for i, (idxT, idxV) in enumerate( skf.split(X_train, y_train, groups=X_train['DT_M']) ):
    month = X_train.iloc[idxV]['DT_M'].iloc[0]
    print('Fold',i,'withholding month',month)
    print(' rows of train =',len(idxT),'rows of holdout =',len(idxV))
    clf = xgb.XGBClassifier(
            n_estimators=5000,
            max_depth=12,
            learning_rate=0.02,
            subsample=0.8,
            colsample_bytree=0.4,
            missing=-1,
            eval_metric='auc',
            # USE CPU
            #nthread=4,
            tree_method='hist'
            # USE GPU
            #tree_method='gpu_hist' 
        )        
    h = clf.fit(X_train[cols].iloc[idxT], y_train.iloc[idxT], 
                eval_set=[(X_train[cols].iloc[idxV],y_train.iloc[idxV])],
                verbose=100, early_stopping_rounds=200)
    
    oof[idxV] += clf.predict_proba(X_train[cols].iloc[idxV])[:,1]
    preds += clf.predict_proba(X_test[cols])[:,1]/skf.n_splits
        
print('#'*20)
print ('XGB96 OOF CV=',roc_auc_score(y_train,oof))

## **UID Magic**

In [ ]:
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')
        
# LABEL ENCODE
def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')
        
# GROUP AGGREGATION MEAN AND STD
# https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')
                
# COMBINE FEATURES
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')
    
# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

In [ ]:
encode_CB('card1','addr1')
#encode_CB('card1_addr1','P_emaildomain')

X_train['day'] =X_train.TransactionDT / (24*60*60)
X_train['uid'] = X_train.card1_addr1.astype(str)+'_'+np.floor(X_train.day-X_train.D1).astype(str)

X_test['day'] = X_test.TransactionDT / (24*60*60)
X_test['uid'] =X_test.card1_addr1.astype(str)+'_'+np.floor(X_test.day-X_test.D1).astype(str)

In [ ]:
%%time
# FREQUENCY ENCODE UID
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
encode_FE(X_train,X_test,['uid'])
# AGGREGATE 
encode_AG(['TransactionAmt','D4','D9','D10','D15'],['uid'],['mean','std'],fillna=True,usena=True)
# AGGREGATE
encode_AG(['C'+str(x) for x in range(1,15) if x!=3],['uid'],['mean'],X_train,X_test,fillna=True,usena=True)
# AGGREGATE
encode_AG(['M'+str(x) for x in range(1,10)],['uid'],['mean'],fillna=True,usena=True)
# AGGREGATE
encode_AG2(['P_emaildomain','dist1','DT_M','id_02','cents'], ['uid'], train_df=X_train, test_df=X_test)
# AGGREGATE
encode_AG(['C14'],['uid'],['std'],X_test,X_test,fillna=True,usena=True)
# AGGREGATE 
encode_AG2(['C13','V314'], ['uid'], train_df=X_train, test_df=X_test)
# AGGREATE 
encode_AG2(['V127','V136','V309','V307','V320'], ['uid'], train_df=X_train, test_df=X_test)
# NEW FEATURE
X_train['outsider15'] = (np.abs(X_train.D1-X_train.D15)>3).astype('int8')
X_test['outsider15'] = (np.abs(X_test.D1-X_test.D15)>3).astype('int8')
print('outsider15')

In [ ]:
# NORMALIZE D COLUMNS
for i in range(1,16):
    if i in [1,2,3,5,9]: continue
    X_train['D'+str(i)] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60) 

In [ ]:
cols = list(X_train.columns )
cols.remove('TransactionDT')
for c in ['DT_M','day','uid']:
    cols.remove(c)
    


In [ ]:
print('NOW USING THE FOLLOWING',len(cols),'FEATURES.')
np.array(cols)

In [ ]:
X_train

In [ ]:
#X = X.drop(columns = ['id_07','id_21','id_22','id_24' ,'id_25' ,'id_26'])
# CHRIS - TRAIN 75% PREDICT 25%
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

In [ ]:
X_train = X_train.drop(columns=['uid'])

In [ ]:
cols = X_train.columns

In [ ]:
X_train

In [ ]:
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

In [ ]:
cols

In [ ]:
oof = np.zeros(len(X_train))
preds = np.zeros(len(X_test))
skf = GroupKFold(n_splits=6)
for i, (idxT, idxV) in enumerate( skf.split(X_train, y_train, groups=X_train['DT_M']) ):
    month = X_train.iloc[idxV]['DT_M'].iloc[0]
    print('Fold',i,'withholding month',month)
    print(' rows of train =',len(idxT),'rows of holdout =',len(idxV))
    clfz = xgb.XGBClassifier(
            n_estimators=5000,
            max_depth=12,
            learning_rate=0.02,
            subsample=0.8,
            colsample_bytree=0.4,
            missing=-1,
            eval_metric='auc',
            # USE CPU
            #nthread=4,
            tree_method='hist'
            # USE GPU
            #tree_method='gpu_hist' 
        )        
    hz = clfz.fit(X_train[cols].iloc[idxT], y_train.iloc[idxT], 
                eval_set=[(X_train[cols].iloc[idxV],y_train.iloc[idxV])],
                verbose=100, early_stopping_rounds=200)
    
    oof[idxV] += clfz.predict_proba(X_train[cols].iloc[idxV])[:,1]
    preds += clfz.predict_proba(X_test[cols])[:,1]/skf.n_splits
        
print('#'*20)
print ('XGB96 OOF CV=',roc_auc_score(y_train,oof))

feature_imp = pd.DataFrame(sorted(zip(clfz.feature_importances_,cols)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).iloc[:50])
plt.title('XGB95 Most Important Features')
plt.tight_layout()
plt.show()

In [ ]:
#start_time = time.time()
#predicted_X_test = hz.predict_proba(final_test[X.columns])
#print("Time to execute SKLearn: %s" % (time.time() - start_time))

In [ ]:
#predicted_X_test.shape

In [ ]:
#predicted_X_test[:, 1]

In [ ]:
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
final_col = test_transaction['TransactionID']

In [ ]:
final_col

# **Submission**

In [ ]:
my_submission = pd.DataFrame({'TransactionID':final_col, 'isFraud': preds})
# you could use any filename. We choose submission here
my_submission.to_csv('./submission.csv', index=False)
my_submission.head(5)